# Project 6

# Pipeline 1: `Download`

In [1]:
!pwd

/Users/nate_velarde/dsi/dsi_workspace/dsi_project_6


In [2]:
# importing modules
import pandas as pd
import lib.database_module as db
import lib.encoding_module as enc
import lib.wiki_module as wiki

In [3]:
conn, cur = db.connect_to_postgres()

Connected to server joshuacook.me.


## Acquire: Get categories to query from a yaml file

In [4]:
import yaml

In [5]:
with open('data/categories.yml') as yaml_file:
    categories = yaml.load(yaml_file)

In [6]:
categories

{'categories': ['Association football trophies and awards']}

## Acquire: Query Wikipedia by category for pages

In [7]:
responses = []

for cat in categories['categories']:
    response = wiki.query_category(cat)
    responses.append(response)

In [8]:
cat_ids = [response['categoryid'] for response in responses]
cat_ids

['27573876']

In [9]:
# need to write to database -- zipped categories and cat_ids

zipped = zip(categories['categories'], cat_ids)
zipped

[('Association football trophies and awards', '27573876')]

## Acquire: Write category info to database

In [10]:
for cat in zip(categories['categories'], cat_ids):
    print (cat)
    db.create_or_update_category_in_database(cat[1], cat[0])

('Association football trophies and awards', '27573876')
Connected to server joshuacook.me.


In [11]:
pd.DataFrame(responses[0]['pages'])

,pageid,title
0,5167857,Footballer of the Year of Argentina
1,13132344,Austrian Footballer of the Year
2,21388412,Azerbaijani Footballer of the Year
3,398110,Ballon d'Or
4,41498124,Balon de Oro (Mexico)
5,48724374,Best Footballer in Asia
6,10540985,Best Male Soccer Player ESPY Award
7,10541158,Best Soccer Player ESPY Award
8,12086359,Brian Clough Trophy
9,8613807,Bulgarian Footballer of the Year


## Acquire: Query Wikipedia by page id for content

Going to work through an example of getting one page and then generalizing to get category pages into the database

In [12]:
# getting page ID from Pandas table above -- 2nd entry

page_response = wiki.query_page("13132344")
pg_13132344_txt = page_response['text']
pg_13132344_txt

u'The title Austrian Footballer of the Year (APA-Fu\xdfballerwahl) has been awarded in Austria since 1984. The coaches of the Bundesliga clubs elect the player of the year. The election is organized by APA, the leading Austrian Press Agency. Eligible for selection are all players in the Austrian league and all Austrian players playing abroad. Austrian Footballer of the Year See also Austrian Sportspersonality of the year References RSSSF'

In [13]:
page_response.keys()

['pageid', 'summary', 'text', 'html']

In [14]:
db.create_or_update_page_in_database(13132344,
                                     27573876, 
                                     'Austrian Footballer of the Year',
                                     page_response['text'])

Connected to server joshuacook.me.


'OK'

In [15]:
response

{'categoryid': '27573876',
 'pages': [{'pageid': 5167857, 'title': 'Footballer of the Year of Argentina'},
  {'pageid': 13132344, 'title': 'Austrian Footballer of the Year'},
  {'pageid': 21388412, 'title': 'Azerbaijani Footballer of the Year'},
  {'pageid': 398110, 'title': "Ballon d'Or"},
  {'pageid': 41498124, 'title': 'Balon de Oro (Mexico)'},
  {'pageid': 48724374, 'title': 'Best Footballer in Asia'},
  {'pageid': 10540985, 'title': 'Best Male Soccer Player ESPY Award'},
  {'pageid': 10541158, 'title': 'Best Soccer Player ESPY Award'},
  {'pageid': 12086359, 'title': 'Brian Clough Trophy'},
  {'pageid': 8613807, 'title': 'Bulgarian Footballer of the Year'},
  {'pageid': 14361922, 'title': 'CAF Awards'},
  {'pageid': 24463998, 'title': 'CAF Clubs of the 20th Century'},
  {'pageid': 36022606, 'title': 'Caribbean Footballer of the Year'},
  {'pageid': 15261184, 'title': 'Chilean Footballer of the Year'},
  {'pageid': 41201053, 'title': 'CONCACAF Awards'},
  {'pageid': 27141150, 'titl

## Acquire: Write page content to database

In [16]:
response.keys()

['categoryid', 'pages']

In [17]:
response['pages'][0]

{'pageid': 5167857, 'title': 'Footballer of the Year of Argentina'}

In [18]:
# Getting list of pageids and titles from my selected Football Association category

pageid_list = []
title_list = []
counter = 0
for page in response['pages']:
    pageid_list.append(response['pages'][counter]['pageid'])
    title_list.append(response['pages'][counter]['title'])
    counter += 1
    
pageid_list

[5167857,
 13132344,
 21388412,
 398110,
 41498124,
 48724374,
 10540985,
 10541158,
 12086359,
 8613807,
 14361922,
 24463998,
 36022606,
 15261184,
 41201053,
 27141150,
 28564020,
 27354657,
 5772743,
 22255796,
 4349290,
 10930297,
 13128047,
 8490081,
 50322415,
 52401067,
 52401100,
 52401727,
 52401687,
 9729324,
 26674327,
 49976178,
 40420231,
 18532081,
 37023556,
 50523433,
 35424101,
 13194746,
 41495686,
 39679054,
 13325961,
 25784770,
 43099601,
 16271291,
 38039788,
 12876089,
 6573759,
 10235434,
 22252551,
 33634228,
 35425398,
 20591470,
 11335136,
 49302539,
 36217266,
 38094609,
 39508609,
 18689236,
 22247461,
 15189653,
 3126617,
 37731169,
 8262489,
 30284945,
 20493501,
 8181732,
 13422773,
 10235532,
 48802632,
 49976246,
 5686612,
 16654048,
 52401333,
 15362131,
 49751777,
 34216232,
 6501736,
 6498811,
 48974460,
 3714550,
 8383656,
 48865363,
 52469429,
 14641991,
 51179925,
 51179458,
 30605664,
 21579944,
 21608841,
 32512047,
 32512103,
 21595530,
 2160

In [19]:
title_list

['Footballer of the Year of Argentina',
 'Austrian Footballer of the Year',
 'Azerbaijani Footballer of the Year',
 "Ballon d'Or",
 'Balon de Oro (Mexico)',
 'Best Footballer in Asia',
 'Best Male Soccer Player ESPY Award',
 'Best Soccer Player ESPY Award',
 'Brian Clough Trophy',
 'Bulgarian Footballer of the Year',
 'CAF Awards',
 'CAF Clubs of the 20th Century',
 'Caribbean Footballer of the Year',
 'Chilean Footballer of the Year',
 'CONCACAF Awards',
 'Copa Libertadores (trophy)',
 'Copa Sudamericana (trophy)',
 'Czech Footballer of the Year',
 'Czechoslovak Footballer of the Year',
 "Pallone d'Argento",
 'Danish Football Player of the Year',
 'Det Gyldne Bur',
 'Dutch Footballer of the Year',
 "DZFoot d'Or",
 'Enzo Bearzot Award',
 'Esiliiga B Player of the Year',
 'Esiliiga Player of the Year',
 'Estonian Female Footballer of the Year',
 'Estonian Female Young Footballer of the Year',
 'Estonian Footballer of the Year',
 'Estonian Silverball',
 'Estonian Young Footballer of the 

In [20]:
# Zipping pageid_list and title_list together

zipped_title_pageid = zip(pageid_list, title_list)
zipped_title_pageid

[(5167857, 'Footballer of the Year of Argentina'),
 (13132344, 'Austrian Footballer of the Year'),
 (21388412, 'Azerbaijani Footballer of the Year'),
 (398110, "Ballon d'Or"),
 (41498124, 'Balon de Oro (Mexico)'),
 (48724374, 'Best Footballer in Asia'),
 (10540985, 'Best Male Soccer Player ESPY Award'),
 (10541158, 'Best Soccer Player ESPY Award'),
 (12086359, 'Brian Clough Trophy'),
 (8613807, 'Bulgarian Footballer of the Year'),
 (14361922, 'CAF Awards'),
 (24463998, 'CAF Clubs of the 20th Century'),
 (36022606, 'Caribbean Footballer of the Year'),
 (15261184, 'Chilean Footballer of the Year'),
 (41201053, 'CONCACAF Awards'),
 (27141150, 'Copa Libertadores (trophy)'),
 (28564020, 'Copa Sudamericana (trophy)'),
 (27354657, 'Czech Footballer of the Year'),
 (5772743, 'Czechoslovak Footballer of the Year'),
 (22255796, "Pallone d'Argento"),
 (4349290, 'Danish Football Player of the Year'),
 (10930297, 'Det Gyldne Bur'),
 (13128047, 'Dutch Footballer of the Year'),
 (8490081, "DZFoot d'O

## This Code Uploads Pages to Database


In [21]:
# this works! -- uploaded!!

for page, title in zipped_title_pageid:
    page_response = wiki.query_page(str(page))
    db.create_or_update_page_in_database(page, int(cat_ids[0]), title, page_response['text'])


Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server 

## Identify: Provide a detailed analysis of the database schema in your report

Bonus